In [13]:
import json
from datetime import datetime
import sagemaker
from sagemaker.predictor import Predictor
from sagemaker.serializers import IdentitySerializer
from sagemaker.deserializers import JSONDeserializer

# 1) Set your deployed endpoint name
ENDPOINT_NAME = "food-demand"  # e.g., restaurant-demand-predictor-01

# 2) Mappings for categorical features
# ORDER REQUIRED BY YOUR MODEL: [day_of_week, weather, menu_item]
DAY_MAPPING = {"friday": 0, "monday": 1, "saturday": 2, "sunday": 3, "thursday": 4, "tuesday": 5, "wednesday": 6}
WEATHER_MAPPING = {"cloudy": 0, "rainy": 1, "sunny": 2}
DISH_MAPPING = {"biryani": 0, "burger": 1, "pasta": 2, "pizza": 3, "salad": 4, "sandwich": 5}

def get_user_inputs():
    print("\n--- Enter details for forecast ---")
    while True:
        try:
            day = input(f"Day of week ({', '.join(DAY_MAPPING.keys())}): ").lower().strip()
            if day not in DAY_MAPPING:
                print("Invalid day. Try again."); continue

            weather = input(f"Weather ({', '.join(WEATHER_MAPPING.keys())}): ").lower().strip()
            if weather not in WEATHER_MAPPING:
                print("Invalid weather. Try again."); continue

            dish = input(f"Dish ({', '.join(DISH_MAPPING.keys())}): ").lower().strip()
            if dish not in DISH_MAPPING:
                print("Invalid dish. Try again."); continue

            return DAY_MAPPING[day], WEATHER_MAPPING[weather], DISH_MAPPING[dish], dish.capitalize()
        except Exception:
            print("Invalid input. Try again.")

def get_prediction(endpoint_name, day_code, weather_code, dish_code, dish_name):
    # Build request with order [day_of_week, weather, menu_item]
    payload = {
        "instances": [{
            "start": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "target": [0],  # placeholder required by DeepAR for inference
            "cat": [day_code, weather_code, dish_code]
        }],
        "configuration": {
            "num_samples": 500,
            "output_types": ["mean", "quantiles"],
            "quantiles": ["0.1", "0.9"]
        }
    }

    # Optional safety checks to avoid out-of-range errors
    assert 0 <= day_code <= 6, "day_of_week out of range (0–6)"
    assert 0 <= weather_code <= 2, "weather out of range (0–2)"
    assert 0 <= dish_code <= 5, "menu_item out of range (0–5)"

    predictor = Predictor(
        endpoint_name=endpoint_name,
        sagemaker_session=sagemaker.Session(),
        serializer=IdentitySerializer(content_type="application/json"),
        deserializer=JSONDeserializer()
    )

    resp = predictor.predict(json.dumps(payload))
    pred = resp["predictions"][0]

    mean_val = pred["mean"][0]
    low = pred["quantiles"]["0.1"][0]
    high = pred["quantiles"]["0.9"][0]

    # Print integers
    mean_int = int(round(mean_val))
    low_int = int(round(low))
    high_int = int(round(high))

    print(f"\n✅ Forecast for {dish_name}")
    print("-------------------------------")
    print(f"Mean Predicted Sales: {mean_int}")
    print(f"Low Estimate (P10):  {low_int}")
    print(f"High Estimate (P90): {high_int}")

if __name__ == "__main__":
    d, w, dish_code, dish_name = get_user_inputs()
    get_prediction(ENDPOINT_NAME, d, w, dish_code, dish_name)


--- Enter details for forecast ---


Day of week (friday, monday, saturday, sunday, thursday, tuesday, wednesday):  sunday
Weather (cloudy, rainy, sunny):  sunny
Dish (biryani, burger, pasta, pizza, salad, sandwich):  pizza



✅ Forecast for Pizza
-------------------------------
Mean Predicted Sales: 51
Low Estimate (P10):  47
High Estimate (P90): 55
